In [ ]:
from analysing_imaging_data import analysing_imaging_data as aid
from custom_plotting import custom_plotting as cp
from custom_stats import custom_stats
from cell_counting_analysis import cell_counting_analysis as cca

import pandas as pd
from scipy import stats
import scikit_posthocs as sp
from statsmodels import formula
from statsmodels import api
import os


%load_ext autoreload
%autoreload 2

In [ ]:
C0_imgs = cca.create_img_dict_from_folder(load_pattern="../Default_Watershed/C0/*.tiff")
C1_imgs = cca.create_img_dict_from_folder(load_pattern="../Default_Watershed/C1/*.tiff")

C1_lab_imgs = cca.create_labelled_img_dict_from_folder(load_pattern="./C1_label_imgs/*.tiff")

In [ ]:
tidy_df = cca.num_div_denom_measure_region_props_to_tidy_df(
    num_img_dict=C1_imgs,
    denom_img_dict=C0_imgs,
    label_imgs=C1_lab_imgs,
)

In [ ]:
my_pal = cp.define_pallette_tidy(tidy_df["sample_id"])

In [ ]:
cp.tidy_create_strip_box_plot(
    ExpName= aid.exp_analysis_name(),
    x_figSize=cp.determine_fig_width_from_palette(my_pal),
    y_axis_limit=8,
    save_fig=False,
    data=tidy_df,
    y_label=cp.identify_y_axis_label(aid.exp_analysis_name()),
    x="sample_id",
    y="mean_intensity_num_div_denom",
    palette=my_pal,
)

In [ ]:
custom_stats.kruskal_scipy_stats_tidy_df_wrapper(
    tidy_df=tidy_df, indep_var="sample_id", dep_var="mean_intensity_num_div_denom"
)

In [ ]:
sp.posthoc_dunn(
    tidy_df, val_col="mean_intensity_num_div_denom", group_col="sample_id", p_adjust="bonferroni"
)

In [ ]:
tidy_df_group = aid.grouped_tidy_data_summary_stats(
    tidy_df=tidy_df,
    group_col="image_key",
    agg_funcs=["mean", "median"],
    categories=tidy_df["sample_id"].cat.categories,
)

In [ ]:
stat_type = "mean"

cp.tidy_create_swarm_box_plot(
    ExpName=aid.exp_analysis_name() + f"_{stat_type}",
    save_fig=True,
    data=tidy_df_group.query(f"summary_stat == '{stat_type}'"),
    y_label=cp.identify_y_axis_label(aid.exp_analysis_name()),
    x_figSize=cp.determine_fig_width_from_palette(my_pal),
    x="sample_id",
    y="mean_intensity_num_div_denom",
    palette=my_pal,
)

sp.posthoc_ttest(
    tidy_df_group.query(f"summary_stat == '{stat_type}'"),
    val_col="mean_intensity_num_div_denom",
    group_col="sample_id",
    pool_sd=True,
    p_adjust="holm-sidak",
)

In [ ]:
stat_type = "median"

cp.tidy_create_swarm_box_plot(
    ExpName=aid.exp_analysis_name() + f"_{stat_type}",
    save_fig=True,
    data=tidy_df_group.query(f"summary_stat == '{stat_type}'"),
    y_label=cp.identify_y_axis_label(aid.exp_analysis_name()),
    x_figSize=cp.determine_fig_width_from_palette(my_pal),
    x="sample_id",
    y="mean_intensity_num_div_denom",
    palette=my_pal,
)

sp.posthoc_ttest(
    tidy_df_group.query(f"summary_stat == '{stat_type}'"),
    val_col="mean_intensity_num_div_denom",
    group_col="sample_id",
    pool_sd=True,
    p_adjust="holm-sidak",
)

In [ ]:
tidy_df.to_csv(f"{aid.exp_analysis_name()}_tidy.csv")
tidy_df_group.to_csv(f"{aid.exp_analysis_name()}_per_gut_tidy.csv")

In [ ]:
if os.path.isfile("Tidy_Image_Analysis_Template_csv.ipynb"):
    os.rename(
        "Tidy_Image_Analysis_Template_csv.ipynb",
        f"{aid.exp_analysis_name()}_csv_tidy.ipynb",
    )